In [1]:
!python --version

Python 3.8.10


In [2]:
import torch

print(torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

torch.cuda.empty_cache()

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

True
A100-SXM-80GB


In [3]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 85198045184
free     : 85094236160
used     : 103809024


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch
from datasets import load_dataset, load_from_disk
from trl import SFTTrainer

In [5]:
base_model = "custom-finetuned_mistral-7b-instruct-v02-18"
tokenizer_model = "./mistral-7b-instruct-v02"

In [6]:
device = 'cuda'

In [7]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        device_map="cuda",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(True, True)

In [8]:
model.config.use_cache = True
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
       

In [9]:
sys_prompt = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format. Do not generate anything after the second %%%%%%%.



"""

In [10]:
sys_prompt_implicit = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Use homecontext to understand the devices, and only consider devices mentioned in the homecontext data. Do not generate prediction for irrelavant devices.

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format. Do not generate anything after the second %%%%%%%.



"""

In [11]:
import json
def display(s):
    t = json.loads(s)
    t2 = json.dumps(t,indent=4)
    print(t2)

In [37]:
user_query = """ Turn on the AC """
logging.set_verbosity(logging.CRITICAL)
prompt = sys_prompt + """@@@ """ + user_query + "@@@"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=8000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "identify_device",
            "arguments": {
                "device": "AC"
            }
        },
        {
            "name": "device_turnOn",
            "arguments": {
                "device": "#E1"
            }
        }
    ]
}


In [38]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [39]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac_12345",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "off",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "b1",
            "deviceName": "Blind",
            "deviceType": "blind",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          }
        ]
      }
"""

In [40]:
user_query = """ Its very hot, switch on cooling """
logging.set_verbosity(logging.CRITICAL)
prompt = sys_prompt + context + """@@@ """ + user_query + "@@@"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "device_turnOn",
            "arguments": {
                "device": "ac_12345"
            }
        }
    ]
}


In [41]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [25]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac1",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "off",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "b1",
            "deviceName": "Blind",
            "deviceType": "blind",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Hall",
              "roomId": "r1"
            }
          }
        ]
      }
"""

In [26]:
user_query = """ Its getting hot in here, make the room cooler """
logging.set_verbosity(logging.CRITICAL)
prompt = sys_prompt + context + """@@@ """ + user_query + "@@@"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=8000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "device_turnOn",
            "arguments": {
                "device": "ac1"
            }
        },
        {
            "name": "device_turnOff",
            "arguments": {
                "device": "b1"
            }
        }
    ]
}


In [27]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [47]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac1",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "off",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "rvc1",
            "deviceName": "Robo Vaccum Cleaner",
            "deviceType": "rvc",
            "deviceStatus": "on",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
            },
          {
            "deviceId": "tv1_ddd",
            "deviceName": "Television",
            "deviceType": "tv",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "l1",
            "deviceName": "Light 1",
            "deviceType": "light",
            "deviceStatus": "off",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
        ]
      }
"""

In [48]:
cross_verify = """



Before Outputting the deviceId in the functions. Check whether this deviceId is actually there or not in `homeContext`



"""

In [49]:
user_query = """ I have to take a call, silence please"""
logging.set_verbosity(logging.CRITICAL)
prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "mute_turnOn",
            "arguments": {
                "device": "tv1_ddd"
            }
        },
        {
            "name": "cleaning_pause",
            "arguments": {
                "device": "rvc1"
            }
        },
        {
            "name": "airConditioningMode_setValue",
            "arguments": {
                "device": "ac1",
                "mode": "WindFree"
            }
        }
    ]
}


In [50]:
result[0]['generated_text']

'<s>[INST] \n\nYou are a smart home IOT agent.\nYou will be given a user command enclosed within @@@ Task @@@.\n\nYou have to generate step by step execution plan for the given user command in json format enclosed between !!!:\n!!!\n%%%%%%%\n{\n\n    "tool_calls": [\n        {\n            "function_name": "function 1",\n            "arguments": \n                {\n                    "argument_name 1": "argument value"\n                }\n         },\n         {\n            "function_name": "function 2",\n            "arguments": \n                {\n                    "argument_name 1": "argument value",\n                    "argument_name 2": "argument value"\n                }\n        }\n    ]\n}\n%%%%%%%\n!!!\nKeywords are defined as below:\n\ntool_calls: List of tool call objects, where each tool call object contains:  \n    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more 

In [42]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac1",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "tv1_ddd",
            "deviceName": "Television",
            "deviceType": "tv",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "l1",
            "deviceName": "Light 1",
            "deviceType": "light",
            "deviceStatus": "on",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
        }
        ]
      }
"""

In [45]:
user_query = """Create a soothing environment"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "media_play",
            "arguments": {
                "device": "tv1_ddd",
                "appName": "Youtube",
                "title": "soothing music"
            }
        },
        {
            "name": "airConditioningMode_setValue",
            "arguments": {
                "device": "ac1",
                "mode": "WindFree"
            }
        },
        {
            "name": "colorMode_setValue",
            "arguments": {
                "device": "l1",
                "mode": "soothing"
            }
        }
    ]
}


In [46]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [54]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac1",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "off",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "tv1_ddd",
            "deviceName": "Television",
            "deviceType": "tv",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "l1",
            "deviceName": "Light 1",
            "deviceType": "light",
            "deviceStatus": "off",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
            }
        ]
      }
"""

In [55]:
user_query = """ Turn on all the devices in the living room except Telivision"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "device_turnOn",
            "arguments": {
                "device": "ac1"
            }
        },
        {
            "name": "device_turnOn",
            "arguments": {
                "device": "l1"
            }
        }
    ]
}


In [56]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [57]:
user_query = """I am feeling dull, change the environment"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "media_play",
            "arguments": {
                "device": "tv1_ddd",
                "appName": "Youtube",
                "title": "funny videos"
            }
        },
        {
            "name": "cleaning_pause",
            "arguments": {
                "device": "rvc1"
            }
        },
        {
            "name": "airConditioningMode_setValue",
            "arguments": {
                "device": "ac1",
                "mode": "auto"
            }
        },
        {
            "name": "colorMode_setValue",
            "arguments": {
                "device": "l1",
                "mode": "dynamic"
            }
        }
    ]
}


In [58]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [59]:
context = """
{
        "homecontext": 
        [
          {
            "deviceId": "ac1",
            "deviceName": "AC",
            "deviceType": "ac",
            "deviceStatus": "off",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "rvc1",
            "deviceName": "Robo Vaccum Cleaner",
            "deviceType": "rvc",
            "deviceStatus": "on",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
            },
          {
            "deviceId": "tv1_ddd",
            "deviceName": "Television",
            "deviceType": "tv",
            "deviceStatus": "on",
            "location": 
            {
              "locationName": "house",
              "locationId": "l1",
              "roomName": "Living Room",
              "roomId": "r1"
            }
          },
          {
            "deviceId": "l1",
            "deviceName": "Light 1",
            "deviceType": "light",
            "deviceStatus": "off",
            "location": {
                "locationName": "house",
                "locationId": "l1",
                "roomName": "Living Room",
                "roomId": "r1"
            }
        ]
      }
"""

In [60]:
user_query = """I want to have candle light dinner with wife"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "media_play",
            "arguments": {
                "device": "tv1_ddd",
                "appName": "Youtube",
                "title": "romantic music"
            }
        },
        {
            "name": "cleaning_pause",
            "arguments": {
                "device": "rvc1"
            }
        },
        {
            "name": "airConditioningMode_setValue",
            "arguments": {
                "device": "ac1",
                "mode": "WindFree"
            }
        },
        {
            "name": "colorMode_setValue",
            "arguments": {
                "device": "l1",
                "mode": "cozy"
            }
        }
    ]
}


In [61]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [65]:
user_query = """I want to have dinner with my kids"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + context + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "media_play",
            "arguments": {
                "device": "tv1_ddd",
                "appName": "Youtube",
                "title": "dinner time songs"
            }
        },
        {
            "name": "cleaning_pause",
            "arguments": {
                "device": "rvc1"
            }
        },
        {
            "name": "airConditioningMode_setValue",
            "arguments": {
                "device": "ac1",
                "mode": "WindFree"
            }
        },
        {
            "name": "colorMode_setValue",
            "arguments": {
                "device": "l1",
                "mode": "cozy"
            }
        }
    ]
}


In [66]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [71]:
user_query = """Set the light brightness to 50%"""
logging.set_verbosity(logging.CRITICAL)

prompt = sys_prompt_implicit + cross_verify + """@@@ """ + user_query + "@@@" + ""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
display(result[0]['generated_text'].split('%%%%%%%')[5])

{
    "type": "function",
    "function": [
        {
            "name": "identify-device",
            "arguments": {
                "device": "light"
            }
        },
        {
            "name": "brightness-increase",
            "arguments": {
                "device": "#E1",
                "brightness": 50
            }
        }
    ]
}
